In [1]:
from pathlib import Path
from typing import Optional

# ========== CONFIGURATION (EDIT ONLY THIS CELL) ==========
# DATA_SOURCE options: 'local' (read local parquet under DATA_DIR) or 'hf' (read via HuggingFace URI)
DATA_SOURCE = 'hf'  # set to 'hf' to read from HF dataset URIs

# For local mode: set DATA_DIR to path containing parquet files or leave None to auto-detect
DATA_DIR: Optional[Path] = None

# For HF mode: base URI prefix for huggingface dataset parquet files
HF_BASE = 'hf://datasets/hao-li/AIDev'

# Hard-coded filter per request: do not change
MIN_STARS = 500  # filter: repos with stars >= 500 (hard-coded)

AGENTS = ['Claude_Code','Copilot','Cursor','Devin','OpenAI_Codex']

# Bot list (taken from Untitled1.ipynb)
BOT_LIST = [
  'copilot-swe-agent[bot]',
  'cursor[bot]',
  'gemini-code-assist[bot]',
  'copilot-pull-request-reviewer[bot]',
  'coderabbitai[bot]',
  'ellipsis-dev[bot]',
  'greptile-apps[bot]',
  'entelligence-ai-pr-reviews[bot]',
  'Copilot',
  'github-advanced-security[bot]',
]

# Short note: Change DATA_SOURCE / DATA_DIR / HF_BASE here to run in different environments.

In [3]:
from pathlib import Path
from typing import Optional


REQUIRED_FILES = ['pull_request.parquet', 'repository.parquet']

def is_dataset_dir(p: Path) -> bool:
    return all((p / f).is_file() for f in REQUIRED_FILES)


def find_dataset_dir(data_dir_hint: Optional[Path] = None) -> Optional[Path]:
    candidates = []
    if data_dir_hint:
        candidates.append(Path(data_dir_hint))
    candidates.extend([Path('.'), Path('..'), Path('../..')])

    for cand in candidates:
        cand = Path(cand)
        # 1) exact match
        if is_dataset_dir(cand):
            return cand.resolve()
        # 2) immediate subdirectories
        if cand.is_dir():
            for sub in cand.iterdir():
                if sub.is_dir() and is_dataset_dir(sub):
                    return sub.resolve()
    return None

# Setup depending on DATA_SOURCE
if DATA_SOURCE == 'local':
    BASE = find_dataset_dir(DATA_DIR)
    if BASE is None:
        raise FileNotFoundError(
            "Dataset directory could not be automatically detected.\n"
            "Set the DATA_DIR environment variable or explicitly set DATA_DIR in the top configuration cell.\n"
            "Example: DATA_DIR = Path('/path/to/dataset_dir')"
        )
    REPO_ROOT = BASE.parent
    OUT_DIR = REPO_ROOT / 'replicationPackage' / 'outputs'
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    print('Dataset dir detected:', relpath_to_repo(BASE, REPO_ROOT))
else:
    # HF mode: no local BASE required
    BASE = None
    REPO_ROOT = Path('.')
    OUT_DIR = REPO_ROOT / 'replicationPackage' / 'outputs'
    OUT_DIR.mkdir(parents=True, exist_ok=True)
    print('Using HF dataset base:', HF_BASE)

# Friendly relative-path printer (avoid showing absolute paths)
def relpath_to_repo(p: Path, repo_root: Optional[Path] = None) -> str:
    try:
        if repo_root is None:
            repo_root = p
        return Path(p).relative_to(repo_root).as_posix()
    except Exception:
        return Path(p).as_posix()

# Unified reader (calls HF URI when requested)

def read_parquet_file(name: str, columns=None):
    import pandas as _pd
    if DATA_SOURCE == 'hf':
        uri = f"{HF_BASE}/{name}"
        return _pd.read_parquet(uri, columns=columns)
    else:
        return _pd.read_parquet(BASE / name, columns=columns)


Using HF dataset base: hf://datasets/hao-li/AIDev


In [4]:
import pandas as pd

# Load minimal columns only (fast + reproducible)
repos = read_parquet_file('repository.parquet', columns=['id','stars'])
prs = read_parquet_file(
    'pull_request.parquet',
    columns=['id','repo_id','agent','user','state','merged_at','html_url']
)
comments = read_parquet_file('pr_comments.parquet', columns=['pr_id','user_type'])

print('repos:', repos.shape)
print('prs:', prs.shape)
print('comments:', comments.shape)

repos: (2807, 2)
prs: (33596, 7)
comments: (39122, 2)


In [72]:
# Step 1: Universe selection (paper-aligned): repos with stars>=500, and closed PRs
repo_ids = set(repos.loc[repos['stars'] >= MIN_STARS, 'id'].astype(int).tolist())
print('repos with stars>=500:', len(repo_ids))

selected = prs[(prs['repo_id'].astype(int).isin(repo_ids)) & (prs['state'] == 'closed')].copy()
selected_ids = set(selected['id'].astype(int).tolist())
print('selected closed agentic PRs:', len(selected_ids))

repos with stars>=500: 1479
selected closed agentic PRs: 11048


In [73]:


bot_assigned = selected[selected['user'].isin(BOT_LIST)].copy()
print('bot_assigned (author in BOT_LIST):', len(bot_assigned))

user_commented_pr_ids = set(
    comments.loc[comments['user_type'] == 'User', 'pr_id']
    .dropna().astype(int).unique().tolist()
)

excluded = bot_assigned[~bot_assigned['id'].astype(int).isin(user_commented_pr_ids)].copy()
excluded_ids = set(excluded['id'].astype(int).tolist())

kept_ids = selected_ids - excluded_ids
print('excluded (bot_assigned & no User comments):', len(excluded_ids))
print('kept:', len(kept_ids))

bot_assigned (author in BOT_LIST): 2681
excluded (bot_assigned & no User comments): 1249
kept: 9799


In [74]:
kept = selected[~selected['id'].astype(int).isin(excluded_ids)].copy()
kept['is_merged'] = kept['merged_at'].notna()

_table = (
    kept.groupby('agent')
    .agg(total=('id','count'), merged=('is_merged','sum'))
    .reset_index()
)
_table['rejected'] = _table['total'] - _table['merged']

order = {a: i for i, a in enumerate(AGENTS)}
table1 = _table[_table['agent'].isin(AGENTS)].copy()
table1['__order'] = table1['agent'].map(order)
table1 = table1.sort_values('__order').drop(columns=['__order']).reset_index(drop=True)

try:
    display(table1)
except NameError:
    print(table1.to_string(index=False))

print('TOTAL', int(table1['total'].sum()), int(table1['merged'].sum()), int(table1['rejected'].sum()))

,agent,total,merged,rejected
0,Claude_Code,213,130,83
1,Copilot,1429,839,590
2,Cursor,788,563,225
3,Devin,3380,1813,1567
4,OpenAI_Codex,3989,2834,1155


TOTAL 9799 6179 3620


In [75]:
# Print results only (no file writes)
try:
    display(table1)
except NameError:
    print(table1.to_string(index=False))

print('TOTAL', int(table1['total'].sum()), int(table1['merged'].sum()), int(table1['rejected'].sum()))
print('Excluded count:', len(excluded_ids))
print('Kept count:', len(kept_ids))
print('Sample excluded ids (first 10):', sorted(excluded_ids)[:10])
print('Sample kept ids (first 10):', sorted(kept_ids)[:10])

,agent,total,merged,rejected
0,Claude_Code,213,130,83
1,Copilot,1429,839,590
2,Cursor,788,563,225
3,Devin,3380,1813,1567
4,OpenAI_Codex,3989,2834,1155


TOTAL 9799 6179 3620
Excluded count: 1249
Kept count: 9799
Sample excluded ids (first 10): [3074526770, 3074531119, 3074557301, 3074635096, 3074652778, 3074763951, 3074773765, 3074774643, 3074779290, 3074784299]
Sample kept ids (first 10): [2756921963, 2757103560, 2757124156, 2757125491, 2757179026, 2757674020, 2757829316, 2758172742, 2758200405, 2758379503]
